In [5]:
import torch
import torch.nn as nn

class ConvNet(nn.Module):

    class Block(nn.Module):
        def __init__(self, in_channels, out_channels, stride):
            super().__init__()
            kernel_size = 3
            padding = (kernel_size - 1) // 2

            self.block_model = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.GELU(),
                nn.Conv2d(out_channels, out_channels, kernel_size, 1, padding, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.GELU(),
                nn.Conv2d(out_channels, out_channels, kernel_size, 1, padding, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.GELU(),
            )

            # Residual connection
            if in_channels != out_channels or stride != 1:
                self.skip = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            else:
                self.skip = nn.Identity()

        def forward(self, x):
            return self.block_model(x) + self.skip(x)

    def __init__(self, channels_l0=64, n_blocks=3, num_classes=8):
        super().__init__()
        cnn_layers = [
            nn.Conv2d(in_channels=3, out_channels=channels_l0, kernel_size=11, stride=2, padding=(11 - 1) // 2, bias=False),
            nn.BatchNorm2d(channels_l0),
            nn.GELU(),
        ]

        c1 = channels_l0
        for _ in range(n_blocks):
            c2 = c1 * 2
            cnn_layers.append(self.Block(c1, c2, stride=2))
            c1 = c2

        cnn_layers.append(nn.AdaptiveAvgPool2d((1, 1)))  # Global Average Pooling
        self.cnn = nn.Sequential(*cnn_layers)
        self.classifier = nn.Linear(c1, num_classes)

    def forward(self, x):
        x = self.cnn(x)
        x = x.view(x.size(0), -1)  # flatten for the classifier
        x = self.classifier(x)
        return x

In [6]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# from  .model import ConvNet  # make sure your model class is saved in this file

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths
data_dir = "/Users/shubhjavia/Desktop/MSAI/SP'25/AI in Health/High Risk Tutorial/Teeth or Dental Image Dataset"  # update this to your local path

# Transforms
transform = transforms.Compose([
    transforms.Resize((512, 512)),   # Resize to make inputs consistent
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# Datasets and Dataloaders
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# Model, loss, optimizer
model = ConvNet(channels_l0=64, n_blocks=3, num_classes=8).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

# Training Loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        correct_preds += predicted.eq(labels).sum().item()
        total += labels.size(0)

    train_accuracy = 100. * correct_preds / total
    avg_loss = running_loss / total

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            val_correct += predicted.eq(labels).sum().item()
            val_total += labels.size(0)
    val_accuracy = 100. * val_correct / val_total

    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {avg_loss:.4f} Train Acc: {train_accuracy:.2f}% Val Acc: {val_accuracy:.2f}%")


Epoch 1/5:   1%|▏         | 3/240 [03:45<4:57:24, 75.29s/it]


KeyboardInterrupt: 